In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from datetime import datetime
import time

In [2]:
df2 = pd.read_excel('enatjus-RaspagemEstadosPequenos-230803-Medicamentos.xlsx')

In [3]:
# Converta a coluna de datas para o formato datetime (assumindo que a coluna com as datas é chamada 'data')
df2['Data'] = pd.to_datetime(df2['Data'], format='%d/%m/%Y %H:%M')

# Defina o período de tempo desejado
start_date = '2022-01-01'
end_date = '2022-12-31'

# Filtre o DataFrame com base no intervalo de datas
df = df2[(df2['Data'] >= start_date) & (df2['Data'] <= end_date)]

In [8]:
df

,Estado,Nº de Processos,Id do parecer,Data,Tecnologia,CID,NAT Responsável,Status,Link do Site,Link do PDF
116,AC - Acre,249.0,107317.0,2022-12-28 15:14:00,Medicamento,C34 - Neoplasia maligna dos brônquios e dos pu...,AC,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...
118,AC - Acre,249.0,106878.0,2022-12-22 15:55:00,Medicamento,M80 - Osteoporose com fratura patológica,AC,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...
123,AC - Acre,249.0,103721.0,2022-12-12 17:30:00,Medicamento,F90.0 - Distúrbios da atividade e da atenção,AC,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...
124,AC - Acre,249.0,103279.0,2022-12-12 16:47:00,Medicamento,F84.0 - Autismo infantil,AC,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...
125,AC - Acre,249.0,108116.0,2022-12-08 11:18:00,Medicamento,J84 - Outras doenças pulmonares intersticiais,AC,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...
...,...,...,...,...,...,...,...,...,...,...
5005,TO - Tocantins,313.0,68292.0,2022-03-16 16:47:00,Medicamento,F84 - Transtornos globais do desenvolvimento,TO,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...
5006,TO - Tocantins,313.0,68272.0,2022-03-16 16:05:00,Medicamento,I50 - Insuficiência cardíaca,TO,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...
5007,TO - Tocantins,313.0,65764.0,2022-02-22 17:20:00,Medicamento,G40 - Epilepsia,TO,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...
5008,TO - Tocantins,313.0,60046.0,2022-01-10 17:50:00,Medicamento,N04 - Síndrome nefrótica,TO,Finalizado,https://www.cnj.jus.br/e-natjus/notaTecnica-da...,https://www.cnj.jus.br/e-natjus/notaTecnica-da...


In [5]:
infos_nota = []

for link in df['Link do Site'][780:]:
    print(link)
    requisicao = requests.get(link, timeout=30)
    requisicao_bs = bs(requisicao.content)
    nota = requisicao_bs.find('div', id='conteudo')
    
    tabela1 = nota.find('table').find('tbody').find_all('td')
    nota_id = tabela1[0].get_text().strip()
    tecnologia = tabela1[1].get_text().strip()
    conclusao = tabela1[2].get_text().strip()
    status = tabela1[3].get_text().strip()
    
    tabela2 = nota.find('div', id='formParecer').find_all('div') # vai até 161
    paciente_idade = tabela2[1].find('input')['value'].strip()
    paciente_genero = tabela2[3].find('input', checked=True).find_parent().text.strip()
    paciente_cidade = tabela2[7].find('select', id='txtCidade').find('option').text.strip()
    advogado_instituicao = tabela2[14].find('option', selected=True).text.strip() if tabela2[14].find('option', selected=True) else 'NA'
    processo_justica = tabela2[17].find('option', selected=True).text.strip()
    processo_vara = tabela2[19].find('input')['value'].strip()
    
    diagnostico_cid = tabela2[23].find('select', id='txtCid').find('option').text.strip()
    diagnostico_nome = tabela2[25].find('input')['value'].strip()
    
    tecnologia_tipo = tabela2[31].find('option', selected=True).text.strip()
    tecnologia_registro = tabela2[33].find('option', selected=True).text.strip() if tabela2[33].find('option', selected=True) else 'NA'
    tecnologia_comercial = tabela2[37].find('option').text.strip() if tabela2[37].find('option') else "NA"
    tecnologia_principio = tabela2[39].find('option').text.strip() if tabela2[39].find('option') else 'NA'
    tecnologia_viaAdministracao = tabela2[45].find('input')['value'].strip() if tabela2[45].find('input')['value'].strip() else 'NA'
    tecnologia_posologia = tabela2[47].find('textarea').text.strip()
    tecnologia_continuo = tabela2[49].find('option', selected=True).text.strip() if tabela2[49].find('option', selected=True) else 'NA'
    tecnologia_offlabel = tabela2[55].find('option', selected=True).text.strip() if tabela2[55].find('option', selected=True) else 'NA'
    tecnologia_PCDT = tabela2[57].find('option', selected=True).text.strip() if tabela2[57].find('option', selected=True) else 'NA'
    tecnologia_incorporacao = tabela2[59].find('option', selected=True).text.strip()
    tecnologia_ondeSUS = tabela2[61].find('option', selected=True).text.strip() if tabela2[61].find('option', selected=True) else 'NA'
    tecnologia_oncologico = tabela2[63].find('option', selected=True).text.strip() if tabela2[63].find('option', selected=True) else 'NA'
    
    outrasTec_nome = tabela2[66].find('input')['value'].strip()
    outrasTec_generico = tabela2[70].find('option', selected=True).text.strip() if tabela2[70].find('option', selected=True) else 'NA'
    outrasTec_similar = tabela2[72].find('option', selected=True).text.strip() if tabela2[72].find('option', selected=True) else 'NA'
    
    evidencias_CONITEC = tabela2[111].find('option', selected=True).text.strip()
    
    conclusao_resultado = tabela2[116].find('option', selected=True).text.strip()
    conclusao_evidencias = tabela2[120].find('option', selected=True).text.strip()
    conclusao_urgencia = tabela2[123].find('option', selected=True).text.strip()
    conclusao_anexos = 'https://www.cnj.jus.br/e-natjus/' + tabela2[137].find('a')['href'] if tabela2[137].find('a')['href'] else 'NA'
    
    infos_nota.append([link, nota_id, tecnologia, conclusao, status, paciente_idade, paciente_genero, paciente_cidade,
                       advogado_instituicao, processo_justica, processo_vara, diagnostico_cid, diagnostico_nome,
                       tecnologia_tipo, 
                       tecnologia_registro, tecnologia_comercial, tecnologia_principio, tecnologia_viaAdministracao,
                       tecnologia_posologia, tecnologia_continuo, tecnologia_offlabel, tecnologia_PCDT,
                       tecnologia_incorporacao, tecnologia_ondeSUS, tecnologia_oncologico, outrasTec_nome, outrasTec_generico,
                       outrasTec_similar, evidencias_CONITEC, conclusao_resultado, conclusao_evidencias, conclusao_urgencia,
                       conclusao_anexos])
    
    time.sleep(3)

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=110383
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=107559
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108362
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108362
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=110054
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108569
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108786
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108786
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108786
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108786
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108701
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=108493
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=107438
https://www.cnj.jus.br/e-

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86740
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86740
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86740
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86740
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86740
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86740
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86740
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86156
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=84202
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=85342
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=85342
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=85342
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=85342
https://www.cnj.jus.br/e-natjus/notaTe

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=68622
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=67932
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=67932
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=67932
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=67932
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=67932
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=67932
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=68833
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=68833
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=69195
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=67933
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=66836
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=66667
https://www.cnj.jus.br/e-natjus/notaTe

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=96890
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=96890
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=96890
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=96890
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=92907
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=93577
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=94317
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=93869
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=94611
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=95064
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=93486
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=93067
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=97178
https://www.cnj.jus.br/e-natjus/notaTe

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=77607
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80020
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=82215
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=81261
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=76972
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79542
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80170
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79313
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79313
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=78795
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=78132
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79137
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79120
https://www.cnj.jus.br/e-natjus/notaTe

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=63177
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=61663
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=59097
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=61411
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=61011
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=62052
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=61778
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=61771
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=61758
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=60665
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=59600
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=60842
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=60238
https://www.cnj.jus.br/e-natjus/notaTe

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=95711
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=95059
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=100248
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=100105
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=95891
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=98848
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=94780
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=88086
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=87361
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=93697
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=96420
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86320
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=86320
https://www.cnj.jus.br/e-natjus/nota

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=74229
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=75093
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=73879
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=73512
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=73442
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=73251
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=73251
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=72657
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=72015
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=71985
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=72143
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=72638
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=71758
https://www.cnj.jus.br/e-natjus/notaTe

https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80344
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80319
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80293
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80284
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80284
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80278
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80269
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80119
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=80098
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79929
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79921
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79726
https://www.cnj.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=79703
https://www.cnj.jus.br/e-natjus/notaTe

In [6]:
notas_df = pd.DataFrame(infos_nota, columns=['Link', 'Nota Id', 'Tecnologia', 'Conclusão', 'Status', 'Idade do paciente', 'Gênero do paciente',
                    'Cidade do paciente', 'Autor está representado por', 'Justiça do processo', 'Vara do processo', 'CID', 
                    'Diagnóstico', 'Tipo de Tecnologia', 'Registro na ANVISA?', 'Nome comercial', 'Princípio ativo', 'Via de administração',
                    'Posologia', 'Uso contínuo?', 'Indicação em conformidade com a aprovada no registro?',
                    'Previsto em Protocolo Clínico e Diretrizes Terapêuticas do Min. da Saúde para a situação clínica do demandante?',
                    'O medicamento está inserido no SUS?', 'Se sim, em qual lista?', 'Oncológico?', 
                    'Descrever as opções disponíveis no SUS e/ou Saúde Suplementar', 'Existe genérico?', 'Existe similar?',
                    'Recomendações da CONITEC para a situação clínica do demandante', 'Conclusão Justificada',
                    'Há evidências científicas?', 'Justifica-se a alegação de urgência, conforme definição de Urgência e Emergência do CFM?',
                    'Link anexo'])

In [7]:
agora = datetime.now().strftime("%Y%m%d%H%M%S")
nome_arquivo = "enatjus-NotaTecnicaCompleta-EstadosPequenos-2022-" + agora + ".xlsx"
notas_df.to_excel(nome_arquivo, index=False)